<a href="https://colab.research.google.com/github/joseph7104/-1INF46-Plan_Compras_Produccion/blob/master/notebooks/preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
%cd "./-1INF46-Plan_Compras_Produccion"

Cloning into '-1INF46-Plan_Compras_Produccion'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 2.71 MiB | 9.92 MiB/s, done.


UsageError: option -1 not recognized ( allowed: "qb" )


In [3]:
import os
RAW_DIR  = "data/raw"
PROC_DIR = "data/processed"
#Verificamos la existencia de los archivos
print("RAW:", os.listdir(RAW_DIR))


RAW: ['VENTAS.csv', 'Calendario.csv', 'INVENTARIO_MOVIMIENTOS.csv', 'PRODUCCIÓN.csv', 'INVENTARIO_ENTRADA_LOTES.csv', 'INVENTARIO_DIARIO.csv', 'RECETAS.csv']


In [17]:
import pandas as pd
import numpy as np

# rutas (las tuyas)
VENTAS_CSV      = f"{RAW_DIR}/VENTAS.csv"
CALENDARIO_CSV  = f"{RAW_DIR}/Calendario.csv"

# --- Carga ---
ventas = pd.read_csv(VENTAS_CSV, parse_dates=['Fecha'], dayfirst=True, keep_default_na=False)
cal    = pd.read_csv(CALENDARIO_CSV, parse_dates=['fecha'], dayfirst=True, keep_default_na=False)

# normaliza fechas (robusto)
ventas['Fecha'] = pd.to_datetime(ventas['Fecha'], errors='coerce', format='%Y-%m-%d')
cal['fecha']    = pd.to_datetime(cal['fecha'],    errors='coerce', format='%Y-%m-%d')

# --- Normaliza nombres desde tu plantilla ---
ventas = ventas.rename(columns={
    'Producto/Plato vendido':'plato',
    'Cantidad':'cantidad',
    'Precio total':'monto_total'
})
assert {'Fecha','plato','cantidad','monto_total'}.issubset(ventas.columns), "Revisa columnas en VENTAS.csv"

# --- Agregación diaria por plato ---
ventas_dia = (ventas.groupby(['Fecha','plato'], as_index=False)
                     .agg(cantidad=('cantidad','sum'),
                          monto_total=('monto_total','sum'))
                     .rename(columns={'Fecha':'fecha'}))

# --- Calendario: solo columnas útiles ---
use_cols = ['fecha'] + [c for c in ['feriado','evento','clima'] if c in cal.columns]
cal = cal[use_cols].drop_duplicates('fecha')

# --- Merge ---
df = ventas_dia.merge(cal, on='fecha', how='left')

# --- Features de fecha ---
df['anio'] = df['fecha'].dt.year
df['mes']  = df['fecha'].dt.month
df['dia']  = df['fecha'].dt.day
df['dow']  = df['fecha'].dt.dayofweek           # 0=lun … 6=dom
df['fin_de_semana'] = (df['dow']>=5).astype(int)

# --- Dummies de TODOS los eventos (dinámico) ---
if 'evento' in df.columns:
    eventos_unicos = (
        df['evento']
        .astype(str)
        .str.strip()
        .replace({'': np.nan, 'nan': np.nan})
        .dropna()
        .unique()
    )

    # crea una dummy por cada evento único
    for ev in eventos_unicos:
        clean_ev = (
            ev.strip()
              .replace(" ", "_")
              .replace(".", "")
              .replace(",", "")
              .replace("ñ","n")
              .replace("á","a").replace("é","e")
              .replace("í","i").replace("ó","o").replace("ú","u")
        )
        df[f'evento_{clean_ev}'] = df['evento'].str.fullmatch(ev, case=False).astype('Int64')

    # (opcional) guarda el listado de eventos detectados
    try:
        pd.DataFrame({'evento_unico': eventos_unicos}).to_csv(f"{PROC_DIR}/eventos_unicos.csv", index=False)
    except Exception as e:
        print("Aviso: no se pudo guardar eventos_unicos.csv:", e)

# --- Dummies de clima (si existe) ---
if 'clima' in df.columns:
    df = pd.get_dummies(df, columns=['clima'], prefix='clima', dummy_na=False)

# --- Orden de columnas (manteniendo tus nombres) ---
base = ['fecha','plato','cantidad','monto_total','anio','mes','dia','dow','fin_de_semana']
extra = [c for c in df.columns if c.startswith('evento_') or c.startswith('clima_') or c=='feriado']
df = df[base + extra].sort_values(['plato','fecha']).reset_index(drop=True)

# --- Guardar ---
#out_daily = f"{PROC_DIR}/dataset_forecast_diario.csv"
#df.to_csv(out_daily, index=False)
print("✅ Guardado — filas:", len(df))
df.head()


✅ Guardado — filas: 10960


,fecha,plato,cantidad,monto_total,anio,mes,dia,dow,fin_de_semana,feriado,...,evento_Fiestas_Patrias,evento_Santa_Rosa_de_Lima,evento_Combate_de_Angamos,evento_Todos_los_Santos,evento_Inmaculada_Concepcion,evento_Navidad,evento_Fin_de_Ano,clima_Lluvioso,clima_Nublado,clima_Soleado
0,2019-01-01,Arroz Chaufa,35,770.0,2019,1,1,1,0,1,...,0,0,0,0,0,0,0,False,False,True
1,2019-01-02,Arroz Chaufa,16,352.0,2019,1,2,2,0,0,...,0,0,0,0,0,0,0,False,True,False
2,2019-01-03,Arroz Chaufa,19,418.0,2019,1,3,3,0,0,...,0,0,0,0,0,0,0,False,False,True
3,2019-01-04,Arroz Chaufa,30,660.0,2019,1,4,4,0,0,...,0,0,0,0,0,0,0,False,True,False
4,2019-01-05,Arroz Chaufa,28,616.0,2019,1,5,5,1,0,...,0,0,0,0,0,0,0,False,True,False


In [18]:
num_rows, num_cols = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 10960
Number of columns: 31


In [19]:
# Asegura orden correcto
df = df.sort_values(['plato','fecha']).reset_index(drop=True)

# ---------- LAGS por plato ----------
# Usa ventas pasadas como predictores
lags = [1, 7, 14, 28]
for L in lags:
    df[f'lag_{L}'] = df.groupby('plato')['cantidad'].shift(L)

# ---------- ROLLING STATS por plato (sin fuga de info) ----------
# Usamos la serie desplazada 1 día para que el rolling no mire el día actual
def add_rolling(group, windows=(7,14,28)):
    s = group['cantidad'].shift(1)
    out = {}
    for w in windows:
        out[f'rolling_mean_{w}'] = s.rolling(w).mean()
        out[f'rolling_std_{w}']  = s.rolling(w).std()
    return pd.DataFrame(out, index=group.index)

roll_feats = df.groupby('plato', group_keys=False).apply(add_rolling)
df = pd.concat([df, roll_feats], axis=1)

# ---------- (Opcional) codificación cíclica de calendario ----------
# Mejora modelos lineales; para árboles no es obligatorio
if 'dow' in df.columns:
    df['dow_sin'] = np.sin(2*np.pi*df['dow']/7)
    df['dow_cos'] = np.cos(2*np.pi*df['dow']/7)
if 'mes' in df.columns:
    df['mes_sin'] = np.sin(2*np.pi*(df['mes']-1)/12)
    df['mes_cos'] = np.cos(2*np.pi*(df['mes']-1)/12)

# ---------- Dataset para modelar (quitar filas con NaN iniciales de lags/rolling) ----------
min_required = ['lag_1','lag_7','lag_14','rolling_mean_7','rolling_std_7']
df_model = df.dropna(subset=[c for c in min_required if c in df.columns]).reset_index(drop=True)

# Guardar (ajusta PROC_DIR si usas otra ruta)
out_features = f"{PROC_DIR}/dataset_forecast_diario.csv"
df_model.to_csv(out_features, index=False)
print("✅ Features añadidas y guardadas en:", out_features)
print("Filas totales:", len(df), "→ filas para modelar (sin NaN de arranque):", len(df_model))


/tmp/ipython-input-2255251862.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  roll_feats = df.groupby('plato', group_keys=False).apply(add_rolling)


✅ Features añadidas y guardadas en: data/processed/dataset_forecast_diario.csv
Filas totales: 10960 → filas para modelar (sin NaN de arranque): 10890


In [20]:
%cd "./-1INF46-Plan_Compras_Produccion"
!git status


[Errno 2] No such file or directory: './-1INF46-Plan_Compras_Produccion'
/content/-1INF46-Plan_Compras_Produccion
On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/processed/dataset_forecast_diario.csv
	data/processed/eventos_unicos.csv

nothing added to commit but untracked files present (use "git add" to track)


In [36]:
# Importar la librería para acceder a los secrets
from google.colab import userdata
import os

# --- Reemplaza estos valores con los tuyos ---
username = "joseph7104"
repository = "-1INF46-Plan_Compras_Produccion"  # <-- Cambia esto por el nombre de tu repositorio
# ---------------------------------------------

# Obtener el token guardado en los secrets de Colab
token = userdata.get('GH_TOKEN')

# Construir la URL del repositorio con el token de autenticación
# El formato es: https://<token>@github.com/<usuario>/<repositorio>.git
repo_url_with_token = f"https://{token}@github.com/{username}/{repository}.git"

# Empujar los cambios al repositorio remoto usando la URL con el token
# Primero, eliminamos el 'origin' viejo para evitar conflictos
!git remote remove origin
# Luego, añadimos el nuevo 'origin' con el token
!git remote add origin {repo_url_with_token}
# Finalmente, hacemos el push
!git push origin master

Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 616.90 KiB | 4.28 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
   582de35..0c34a5c  master -> master
